In [3]:
import pandas as pd
import os
import re
import torch
import shutil

# Set GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [6]:
# Define which dataset you want to run
dataset = "mammogram"
folder = "one_third_step_10%"
seed = 584
training_fixed = "fixed"
Retraining = "Retraining"
split = "70_30"
path = f"/root/workspace/Dynamic_Data_Imputation/v2/{dataset}/{folder}"

config = {
    "adultsample": {"initial": 1000, "step": 100},
    "australian": {"initial": 230, "step": 23},
    "contraceptive": {"initial": 491, "step": 49},
    "credit": {"initial": 218, "step": 22},
    "flare": {"initial": 355, "step": 35},
    "imdb": {"initial": 1510, "step": 151},
    "mammogram": {"initial": 277, "step": 27},
    "thoracic": {"initial": 157, "step": 15},
}

initial = config[dataset]["initial"]
step = config[dataset]["step"]
%run /root/workspace/Dynamic_Data_Imputation/v2/zCode/main.py \
  --path {path} \
  --datasets {dataset} \
  --seeds {seed} \
  --cum_pcts 0.05 0.05 0.1 0.2 \
  --initial {initial} \
  --step {step} \
  --mode all
  
  

 
# #--mode inject \   # Only inject nulls:
# #--mode SENT-I \   # Only run FAISS imputation (expects *_nonimputed.csv files already present):
# #--mode all \      # Do both injection and imputation in sequence:




# #________________________Combining all evaluation files and Copying the files to another directory_______


def combine_csvs(input_dir, output_file):
    all_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(f"_{seed}_imputed_evaluation.csv")]
    combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    output_path = os.path.join(input_dir, output_file)
    combined_df.to_csv(output_path, index=False)
    print(f"Combined CSV saved to {output_path}")

#_________________________________Copying the files to other directories_________________________________

def copy_chunk_csvs(src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    chunk_pattern = re.compile(r'_chunk\d+\.csv$')

    for filename in os.listdir(src_dir):
        if chunk_pattern.search(filename):
            src_file = os.path.join(src_dir, filename)
            dst_file = os.path.join(dst_dir, filename)
            shutil.copy2(src_file, dst_file)
            print(f"Copied: {filename}")

#________________________________________________________________________________________________________


input_dir = f"/root/workspace/Dynamic_Data_Imputation/v2/{dataset}/{folder}"
output_file = f"SENT-I_evaluations_{dataset}_{seed}_{folder}.csv"

destination_directory_1 = f"/root/workspace/IPM-main/Dynamic_data/{dataset}/{split}/{training_fixed}_{folder}"
destination_directory_2 = f"/root/workspace/IPM-main/Dynamic_data/{dataset}/{split}/{Retraining}_{folder}"

# Call the combine function
combine_csvs(input_dir, output_file)

# # Copy to both destinations
copy_chunk_csvs(input_dir, destination_directory_1)
copy_chunk_csvs(input_dir, destination_directory_2)



  # after your %run finishes
import gc, torch
for name in list(globals()):
    obj = globals()[name]
    if hasattr(obj, 'device') and getattr(obj, 'device', None).type == 'cuda':
        del globals()[name]

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()  # if you used multiprocessing



=== Dataset: mammogram ===
  Chunk end-indices: [277, 304, 331, 358, 385, 412, 439, 466, 493, 520, 547, 574, 601, 628, 655, 682, 709, 736, 763, 790, 817, 830]

--- Seed 584 (injection) ---

  Building chunk 1: rows 0…276
  Injecting 5% nulls into new rows
    → Injecting into rows 0–276
    → Column 'A1': injecting 13 nulls
    → Column 'A3': injecting 13 nulls
    → Column 'A4': injecting 13 nulls
    → Column 'A5': injecting 13 nulls
    → Column 'A6': injecting 13 nulls
    Saved: mammogram_1_5_584_nonimputed.csv
    Saved new-chunk CSV: mammogram_1_5_584_nonimputed_chunk1.csv
  Injecting 10% nulls into new rows
    → Injecting into rows 0–276
    → Column 'A1': injecting 14 nulls
    → Column 'A3': injecting 14 nulls
    → Column 'A4': injecting 14 nulls
    → Column 'A5': injecting 14 nulls
    → Column 'A6': injecting 14 nulls
    Saved: mammogram_1_10_584_nonimputed.csv
    Saved new-chunk CSV: mammogram_1_10_584_nonimputed_chunk1.csv
  Injecting 20% nulls into new rows
    → I

/root/.pyenv/versions/tariq/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Loaded embedding model on device: cuda:0

--- Seed 584, 5% → imputation ---

  Processing mammogram_1_5_584_nonimputed
    → FAISS build/update for rows 0–276
Built FAISS GPU index via index_cpu_to_all_gpus
    → FAISS time: 0.18s
    → Imputing 62 rows


      Imputing:   0%|          | 0/62 [00:00<?, ?it/s]

    → Imputation time: 1.14s
    Saved chunked imputed CSV: mammogram_1_5_584_imputed.csv
    Saved cumulative subset CSV: mammogram_1_5_584_imputed.csv
    Saved log JSON: mammogram_1_5_584_imputed.json
    Saved evaluation CSV: mammogram_1_5_584_imputed_evaluation.csv
    Saved evaluation CSV: mammogram_1_5_584_imputed_evaluation.csv

  Processing mammogram_2_5_584_nonimputed
    → FAISS build/update for rows 277–303
    → FAISS time: 0.02s
    → Imputing 5 rows


...

    → Imputation time: 0.50s
    Saved chunked imputed CSV: mammogram_20_40_584_imputed.csv
    Saved cumulative subset CSV: mammogram_20_40_584_imputed.csv
    Saved log JSON: mammogram_20_40_584_imputed.json
    Saved evaluation CSV: mammogram_20_40_584_imputed_evaluation.csv
    Saved evaluation CSV: mammogram_20_40_584_imputed_evaluation.csv

  Processing mammogram_21_40_584_nonimputed
    → FAISS build/update for rows 790–816
    → FAISS time: 0.02s
    → Imputing 22 rows


      Imputing:   0%|          | 0/22 [00:00<?, ?it/s]

    → Imputation time: 0.50s
    Saved chunked imputed CSV: mammogram_21_40_584_imputed.csv
    Saved cumulative subset CSV: mammogram_21_40_584_imputed.csv
    Saved log JSON: mammogram_21_40_584_imputed.json
    Saved evaluation CSV: mammogram_21_40_584_imputed_evaluation.csv
    Saved evaluation CSV: mammogram_21_40_584_imputed_evaluation.csv

  Processing mammogram_22_40_584_nonimputed
    → FAISS build/update for rows 817–829
    → FAISS time: 0.02s
    → Imputing 12 rows


      Imputing:   0%|          | 0/12 [00:00<?, ?it/s]

    → Imputation time: 0.26s
    Saved chunked imputed CSV: mammogram_22_40_584_imputed.csv
    Saved cumulative subset CSV: mammogram_22_40_584_imputed.csv
    Saved log JSON: mammogram_22_40_584_imputed.json
    Saved evaluation CSV: mammogram_22_40_584_imputed_evaluation.csv
    Saved evaluation CSV: mammogram_22_40_584_imputed_evaluation.csv

All done.
Combined CSV saved to /root/workspace/Dynamic_Data_Imputation/v2/mammogram/one_third_step_10%/SENT-I_evaluations_mammogram_584_one_third_step_10%.csv
Copied: mammogram_15_10_94_nonimputed_chunk15.csv
Copied: mammogram_12_10_94_nonimputed_chunk12.csv
Copied: mammogram_19_5_584_nonimputed_chunk19.csv
Copied: mammogram_12_20_94_nonimputed_chunk12.csv
Copied: mammogram_15_40_1234_nonimputed_chunk15.csv
Copied: mammogram_15_40_94_nonimputed_chunk15.csv
Copied: mammogram_18_20_94_nonimputed_chunk18.csv
Copied: mammogram_4_5_1234_nonimputed_chunk4.csv
Copied: mammogram_19_5_1234_nonimputed_chunk19.csv
Copied: mammogram_1_5_1234_nonimputed_c

In [8]:
# #________________________Combining all evaluation files and Copying the files to another directory__________________________________


# def combine_csvs(input_dir, output_file):
#     all_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(f"_{seed}_imputed_evaluation.csv")]
#     combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
#     output_path = os.path.join(input_dir, output_file)
#     combined_df.to_csv(output_path, index=False)
#     print(f"Combined CSV saved to {output_path}")
    
    
# input_dir = f"/root/workspace/Dynamic_Data_Imputation/v2/{dataset}/{folder}"
# output_file = f"SENT-I_evaluations_{dataset}_{seed}_{folder}.csv"
# combine_csvs(input_dir, output_file)


Combined CSV saved to /root/workspace/Dynamic_Data_Imputation/v2/adultsample/one_third_step_10%/SENT-I_evaluations_adultsample_one_third_step_10%.csv


In [9]:
# #_________________________________Copying the files to another directory_________________________________

# def copy_chunk_csvs(src_dir, dst_dir):
#     os.makedirs(dst_dir, exist_ok=True)
#     chunk_pattern = re.compile(r'_chunk\d+\.csv$')

#     for filename in os.listdir(src_dir):
#         if chunk_pattern.search(filename):
#             src_file = os.path.join(src_dir, filename)
#             dst_file = os.path.join(dst_dir, filename)
#             shutil.copy2(src_file, dst_file)
#             print(f"Copied: {filename}")

# input_dir = f"/root/workspace/Dynamic_Data_Imputation/v2/{dataset}/{folder}"
# destination_directory = f"/root/workspace/IPM-main/Dynamic_data/{dataset}/{split}/{training_fixed}_{folder}"
# copy_chunk_csvs(input_dir, destination_directory)


Copied: adultsample_18_20_1234_nonimputed_chunk18.csv
Copied: adultsample_2_10_1234_nonimputed_chunk2.csv
Copied: adultsample_12_5_1234_nonimputed_chunk12.csv
Copied: adultsample_21_5_1234_nonimputed_chunk21.csv
Copied: adultsample_4_10_1234_nonimputed_chunk4.csv
Copied: adultsample_14_20_1234_nonimputed_chunk14.csv
Copied: adultsample_20_5_1234_nonimputed_chunk20.csv
Copied: adultsample_19_10_1234_nonimputed_chunk19.csv
Copied: adultsample_3_40_1234_nonimputed_chunk3.csv
Copied: adultsample_15_40_1234_nonimputed_chunk15.csv
Copied: adultsample_16_20_1234_nonimputed_chunk16.csv
Copied: adultsample_8_40_1234_nonimputed_chunk8.csv
Copied: adultsample_12_10_1234_nonimputed_chunk12.csv
Copied: adultsample_15_10_1234_nonimputed_chunk15.csv
Copied: adultsample_6_40_1234_nonimputed_chunk6.csv
Copied: adultsample_9_10_1234_nonimputed_chunk9.csv
Copied: adultsample_22_20_1234_nonimputed_chunk22.csv
Copied: adultsample_3_10_1234_nonimputed_chunk3.csv
Copied: adultsample_12_40_1234_nonimputed_chu